<a href="https://colab.research.google.com/github/charlesfg/term.ooo-solver/blob/main/term_ooo_cheat_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget  https://raw.githubusercontent.com/charlesfg/term.ooo-solver/main/words5char.json

--2022-02-21 14:25:17--  https://raw.githubusercontent.com/charlesfg/term.ooo-solver/main/words5char.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54270 (53K) [text/plain]
Saving to: ‘words5char.json’

words5char.json     100%[===================>]  53.00K  --.-KB/s    in 0.006s  

2022-02-21 14:25:17 (8.69 MB/s) - ‘words5char.json’ saved [54270/54270]



In [2]:
import json 
with open('words5char.json') as f:
  words5char = json.load(f)

print(f"We have {len(words5char)} words in our dataset")

We have 5427 words in our dataset


In [19]:
import re
import operator
import random

# Model of 5 character words 

vowel_f = {
    "a" : 0.1564,
    "o" : 0.1019,
    "e" : 0.1009,
    "i" : 0.0795,
    "u" : 0.0465,
} 

consonant_f = {
    "r" : 0.0771,
    "s" : 0.0725,
    "m" : 0.0477,
    "t" : 0.0404,
    "c" : 0.0400,
    "l" : 0.0376,
    "n" : 0.0320,
    "d" : 0.0278,
    "p" : 0.0255,
    "g" : 0.0220,
    "v" : 0.0218,
    "b" : 0.0197,
    "f" : 0.0195,
    "h" : 0.0086,
    "j" : 0.0076,
    "z" : 0.0074,
    "x" : 0.0060,
    "q" : 0.0014,
    "k" : 0.0001,
}



"""
Given a set of test characteres it will test if they relates to the word_str
returning trues if this relation holds.

Ex.  check('ab',"colegio","in") ==> False
     check('ab',"colegio","out") ==> True
     check('ab',"colegio","biro") ==> TypeError
"""
def check(test_chars,word_str, relations):
  if relations == "in":
    for i in test_chars:
      if i not in word_str:
        return False
    return True
  if relations == "out":
    for i in test_chars:
      if i in word_str:
        return False
    return True
  raise TypeError("relations should be in/out")


"""
in_word       - characters that must exist in the word
out_of_word   - characters that can't exist in the word
word_regex    - regex describing the word (which characters and in 
      which position), ex. '^.a..s$' for 'patos'
"""
def print_candidates(in_word, out_of_word, word_regex, ret=False):
  candidates = []
  for w in words5char:
    if word_regex:
      if not re.match(word_regex, w):
        continue
    if not check(in_word, w, "in"):
      continue
    if not check(out_of_word, w, "out"):
      continue
    
    candidates.append(w)

  if ret:
    return candidates

  for i in candidates:
    print(i)


def print_candidates_trained(in_word, out_of_word, word_regex):
  iw_l = len(in_word)

  vc = vowel_f.copy()
  cc = consonant_f.copy()

  if iw_l == 5:
    print_candidates(in_word,out_of_word, word_regex)
    return

  for o in out_of_word+in_word:
    if o in vc:
      del vc[o]
      #print("removing ",o)
    if o in cc:
      del cc[o]
      #print("removing ",o)
 
  
  if iw_l == 0:
    tmp_inw = "".join([x[0] for x in sorted(vc.items(), key=operator.itemgetter(1), reverse=True)][0:3])
    tmp_inw += "".join([x[0] for x in sorted(cc.items(), key=operator.itemgetter(1), reverse=True)][0:2])
    print_candidates(tmp_inw, out_of_word, word_regex)    
    return 

  new_chars = ""
  candidates = None
  #print("-",in_word)
  
  for i in range(5-iw_l):    
    char_source = cc
    # Just one vowel per try after the first try
    if i == 0:
      if vc:
        char_source = vc
    tmp_c = [x[0] for x in sorted(char_source.items(), key=operator.itemgetter(1), reverse=True)][0]
    #print(" tmp_c:",tmp_c)
    new_chars += tmp_c
    del char_source[tmp_c]  
  print("-",in_word+new_chars)

  while not candidates:
    new_chars= new_chars[:-1]
    print("Failed! retrying with '{}'".format(in_word+new_chars))
    candidates = print_candidates(in_word+new_chars,out_of_word, word_regex, True)
  
  print(candidates)
    
 
  



In [21]:
#print_candidates_trained("","","")
print_candidates_trained("lai","tmrose","[^rl][^ao][^ts][^ie][^ma]")
#print_candidates_trained("cuom","pseirta","c[^u][^p][^o]m")
#print_candidates("omit","sera","[^s][^e][^r][^a][^o]")

- laiuc
Failed! retrying with 'laiu'
['aludi', 'pulai', 'igual', 'ablui', 'aflui']
